# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOBBIES')]

In [5]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Hobbies/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics and Croston_TSB algorithm

In [6]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [7]:
def Croston_TSB(ts,extra_periods=1,alpha=0.4,beta=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), probability(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
# Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1/(1 + first_occurence)
    f[0] = p[0]*a[0]
                 
    # Create all the t+1 forecasts
    for t in range(0,cols): 
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = beta*(1) + (1-beta)*p[t]  
        else:
            a[t+1] = a[t]
            p[t+1] = (1-beta)*p[t]       
        f[t+1] = p[t+1]*a[t+1]
        
    # Future Forecast
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Set value for parameters

In [ ]:
list_params_alpha = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]
list_params_beta = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [8]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 100
validation_training_ratio = 0.95

# Pipeline for Croston_TSB

In [ ]:
def CrostonTSB_output_all_params(sales_pattern, list_params_alpha, list_params_beta, 
                                 start_train_date, end_train_date, n_pred_days, 
                                 num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params_alpha:
            for j in list_params_beta:
                predictions = list()
                history = [x for x in train_first[num_train_needed*-1:]]
                for t in range(len(train_second)):
                    yhat = Croston_TSB(history, extra_periods=1, alpha = i, beta = j)['Forecast'].iloc[-1]
                    predictions.append(yhat)
                    history.append(train_second[t])
                df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                               'Actual Data': train_second,
                                               'Forecast': predictions,
                                               'Alpha': [i for count_i in range(len(train_second))],
                                               'Beta': [j for count_j in range(len(train_second))]})
                df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [ ]:
start = timer()

pattern_df_result_all_params = CrostonTSB_output_all_params(sales_erratic,
                                                          list_params_alpha,
                                                          list_params_beta,
                                                          start_train_date,
                                                          end_train_date, 
                                                          n_pred_days,
                                                          num_train_needed,
                                                          validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 2.8910969301999936 minutes


In [ ]:
pattern_df_result_all_params

,Product,Actual Data,Forecast,Alpha,Beta
0,HOBBIES_1_004_CA_1_validation,1.0,1.507742,0.1,0.1
1,HOBBIES_1_004_CA_1_validation,4.0,1.483145,0.1,0.1
2,HOBBIES_1_004_CA_1_validation,2.0,1.690001,0.1,0.1
3,HOBBIES_1_004_CA_1_validation,1.0,1.724308,0.1,0.1
4,HOBBIES_1_004_CA_1_validation,1.0,1.672386,0.1,0.1
...,...,...,...,...,...
200794,HOBBIES_1_404_CA_4_validation,2.0,5.935843,0.9,0.9
200795,HOBBIES_1_404_CA_4_validation,9.0,2.396849,0.9,0.9
200796,HOBBIES_1_404_CA_4_validation,14.0,8.339145,0.9,0.9
200797,HOBBIES_1_404_CA_4_validation,6.0,13.433868,0.9,0.9


In [ ]:
pattern_df_result_all_params.to_csv('CrostonTSB_Erratic_All_Parameters_Hyperparameter_Tuning_Data.csv')

# Summay all metrics

In [9]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [ ]:
start = timer()

df_result_metrics_all_params = pattern_df_result_all_params.groupby(['Product', 'Alpha', 'Beta']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.21009079250000012 minutes


In [ ]:
df_result_metrics_all_params

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_004_CA_1_validation,0.1,0.1,0.880760,87.5052,102.7932,57.6798,1.419003,1.610849
1,HOBBIES_1_004_CA_1_validation,0.1,0.2,0.885711,87.9971,103.1566,56.8804,1.426979,1.638715
2,HOBBIES_1_004_CA_1_validation,0.1,0.3,0.890528,88.4756,103.2043,56.5214,1.434739,1.660328
3,HOBBIES_1_004_CA_1_validation,0.1,0.4,0.897079,89.1265,103.9651,57.3852,1.445294,1.675028
4,HOBBIES_1_004_CA_1_validation,0.1,0.5,0.904679,89.8816,105.5344,59.0739,1.457539,1.685504
...,...,...,...,...,...,...,...,...,...
5422,HOBBIES_1_404_CA_4_validation,0.9,0.5,0.860573,79.1375,90.4388,108.8773,6.095726,8.095530
5423,HOBBIES_1_404_CA_4_validation,0.9,0.6,0.869374,79.9468,91.1596,110.5313,6.158067,8.169669
5424,HOBBIES_1_404_CA_4_validation,0.9,0.7,0.882954,81.1956,92.3568,112.5880,6.254256,8.292830
5425,HOBBIES_1_404_CA_4_validation,0.9,0.8,0.899647,82.7307,94.0801,114.8523,6.372498,8.470449


# Check what products has unexpected metrics result

In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
2754,FOODS_1_163_CA_1_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
2755,FOODS_1_163_CA_1_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
2756,FOODS_1_163_CA_1_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
2757,FOODS_1_163_CA_1_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
2758,FOODS_1_163_CA_1_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
20002,FOODS_3_827_CA_1_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
20003,FOODS_3_827_CA_1_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
20004,FOODS_3_827_CA_1_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
20005,FOODS_3_827_CA_1_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
2754,FOODS_1_163_CA_1_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
2755,FOODS_1_163_CA_1_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
2756,FOODS_1_163_CA_1_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
2757,FOODS_1_163_CA_1_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
2758,FOODS_1_163_CA_1_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
20002,FOODS_3_827_CA_1_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
20003,FOODS_3_827_CA_1_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
20004,FOODS_3_827_CA_1_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
20005,FOODS_3_827_CA_1_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
2754,FOODS_1_163_CA_1_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
2755,FOODS_1_163_CA_1_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
2756,FOODS_1_163_CA_1_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
2757,FOODS_1_163_CA_1_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
2758,FOODS_1_163_CA_1_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
20002,FOODS_3_827_CA_1_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
20003,FOODS_3_827_CA_1_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
20004,FOODS_3_827_CA_1_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
20005,FOODS_3_827_CA_1_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
2754,FOODS_1_163_CA_1_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.0,0.0
2755,FOODS_1_163_CA_1_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.0,0.0
2756,FOODS_1_163_CA_1_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.0,0.0
2757,FOODS_1_163_CA_1_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.0,0.0
2758,FOODS_1_163_CA_1_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
20002,FOODS_3_827_CA_1_validation,0.9,0.5,NaN,NaN,NaN,NaN,0.0,0.0
20003,FOODS_3_827_CA_1_validation,0.9,0.6,NaN,NaN,NaN,NaN,0.0,0.0
20004,FOODS_3_827_CA_1_validation,0.9,0.7,NaN,NaN,NaN,NaN,0.0,0.0
20005,FOODS_3_827_CA_1_validation,0.9,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [ ]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [ ]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 4.477611940298507 %
Percentage of unexpected values of WMAPE is: 4.477611940298507 %
Percentage of unexpected values of SMAPE is: 1.4925373134328357 %
Percentage of unexpected values of MAPE is: 4.477611940298507 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [ ]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

MASE      WMAPE       SMAPE        MAPE       MAE      RMSE
Alpha Beta                                                                 
0.1   0.1   0.744911  76.084122   85.047605  100.671159  5.158575  7.163417
      0.2   0.749852  76.605572   86.181339  104.196169  5.201405  7.192134
      0.3   0.753709  77.024020   87.060492  106.747353  5.232429  7.222280
      0.4   0.757881  77.483695   87.914637  108.839377  5.259421  7.257385
      0.5   0.763178  78.065789   88.895745  110.815466  5.292757  7.298995
...              ...        ...         ...         ...       ...       ...
0.9   0.5   0.907062  93.248317   99.542458  131.478405  6.373403  9.019807
      0.6   0.913328  93.883705  100.379905  132.947522  6.418906  9.087594
      0.7   0.920707  94.631617  101.417289  134.513423  6.470104  9.166034
      0.8   0.930083  95.605653  102.964092  136.532209  6.533657  9.257178
      0.9   0.941930  96.861611  105.404906  139.199902  6.610906  9.363881

[81 rows x 6 columns]

In [ ]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha and Beta based on Mean MASE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean WMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean SMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean MAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean MAE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean RMSE is: (0.1, 0.1)


### Get MEDIAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

MASE     WMAPE      SMAPE       MAPE       MAE      RMSE
Alpha Beta                                                              
0.1   0.1   0.713335  74.55300   84.66160   95.45475  4.890833  6.514756
      0.2   0.719584  76.27175   86.36235  100.56085  4.935153  6.485697
      0.3   0.726546  76.94360   87.38090  104.59215  4.946052  6.478261
      0.4   0.734718  77.26490   88.27905  104.79935  4.890111  6.546596
      0.5   0.745096  77.19815   89.11150  105.54455  4.892581  6.661939
...              ...       ...        ...        ...       ...       ...
0.9   0.5   0.900267  92.69810   98.50515  128.54190  6.088008  8.119131
      0.6   0.905488  93.38330   98.78640  130.06465  6.108972  8.219726
      0.7   0.910671  94.80415   98.37635  127.89670  6.158067  8.339934
      0.8   0.918253  95.74795   99.26640  128.80885  6.217288  8.471420
      0.9   0.930863  96.64920  101.14475  133.33785  6.336565  8.613342

[81 rows x 6 columns]

In [ ]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha and Beta based on Median of MASE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of WMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of SMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of MAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of MAE is: (0.1, 0.4)
The optimum Alpha and Beta based on Median of RMSE is: (0.1, 0.3)


# Run Croston model after decide best paramaters

In [10]:
best_alpha = 0.1
best_beta = 0.1

In [11]:
def CrostonTSB_test_data(best_alpha, best_beta, sales_pattern,
                         start_train_date, end_train_date, 
                         n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston_TSB(history, extra_periods=1, alpha=best_alpha, beta = best_beta)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [12]:
start = timer()

pattern_df_result_best_params = CrostonTSB_test_data(best_alpha, best_beta, sales_erratic, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.018198914816666682 minutes


In [13]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [14]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOBBIES_1_004_CA_1_validation,0.776275,53.7421,60.3890,52.9868,1.074842,1.403148
1,HOBBIES_1_008_CA_1_validation,0.607589,91.2891,107.5560,242.7240,8.085608,9.974422
2,HOBBIES_1_008_CA_3_validation,0.783906,95.1978,93.7807,161.9634,6.391853,8.458776
3,HOBBIES_1_014_CA_3_validation,0.698742,102.6124,103.5231,92.1092,2.418722,2.939556
4,HOBBIES_1_015_CA_1_validation,0.775234,63.0243,65.3608,124.0853,4.591773,5.356203
...,...,...,...,...,...,...,...
62,HOBBIES_1_381_CA_4_validation,0.778723,124.8618,129.1735,76.3335,4.013417,4.428690
63,HOBBIES_1_387_CA_3_validation,0.590856,69.8973,79.0122,100.1533,6.590316,8.630302
64,HOBBIES_1_404_CA_1_validation,0.684195,84.8206,81.2743,138.8783,5.210405,6.997099
65,HOBBIES_1_404_CA_2_validation,0.739683,88.3167,92.9821,123.4363,5.803670,8.769604


In [15]:
df_result_final.to_csv('CrostonTSB_Erratic_Test_Data.csv')